In [1]:
import numpy as np
import pandas as pd
import lightgbm as lgb
from sklearn.model_selection import train_test_split
import pandas as pd

In [2]:
status = pd.read_csv("status.csv") 
trip = pd.read_csv("trip.csv") 
station = pd.read_csv("station.csv") 
weather = pd.read_csv("weather.csv") 

In [3]:
ls = []
ls2 = []

for i in range(1226400):
    if len(str(status.iloc[i,2])) < 2 and len(str(status.iloc[i,3])) < 2:
        ls.append(str(status.iloc[i,1]) + '-0' + str(status.iloc[i,2]) + '-0' + str(status.iloc[i,3]))
        ls2.append(str(status.iloc[i,1]) + '0' + str(status.iloc[i,2]) + '0' + str(status.iloc[i,3]))
    elif len(str(status.iloc[i,2])) < 2 and len(str(status.iloc[i,3])) > 1:
        ls.append(str(status.iloc[i,1]) + '-0' + str(status.iloc[i,2]) + '-' + str(status.iloc[i,3]))
        ls2.append(str(status.iloc[i,1]) + '0' + str(status.iloc[i,2]) + str(status.iloc[i,3]))
    elif len(str(status.iloc[i,2])) > 1 and len(str(status.iloc[i,3])) < 2:
        ls.append(str(status.iloc[i,1]) + '-' + str(status.iloc[i,2]) + '-0' + str(status.iloc[i,3]))
        ls2.append(str(status.iloc[i,1]) + str(status.iloc[i,2]) + '0' + str(status.iloc[i,3]))
    else:
        ls.append(str(status.iloc[i,1]) + '-' + str(status.iloc[i,2]) + '-' + str(status.iloc[i,3]))
        ls2.append(str(status.iloc[i,1]) + str(status.iloc[i,2]) + str(status.iloc[i,3]))


In [4]:
status['date'] = pd.to_datetime(ls)

In [5]:
import datetime
import jpholiday

def isBizDay(DATE):
    Date = datetime.date(int(DATE[0:4]), int(DATE[4:6]), int(DATE[6:8]))
    if Date.weekday() >= 5 or jpholiday.is_holiday(Date):
        return 1
    else:
        return 0

ls3 = []

for i in range(1226400):
    ls3.append(isBizDay(ls2[i]))
    

In [6]:
status['holiday'] = ls3

In [7]:
status['weekday'] = status['date'].dt.weekday

In [9]:
ls_rain, ls_fog = [],[]

for i in range(730):
    if weather.iloc[i,-2] == 'Rain':
        ls_rain.append(1)
        ls_fog.append(0)
    elif weather.iloc[i,-2] == 'Fog':
        ls_rain.append(0)
        ls_fog.append(1)
    elif weather.iloc[i,-2] == 'Fog-Rain':
        ls_rain.append(1)
        ls_fog.append(1)
    else:
        ls_rain.append(0)
        ls_fog.append(0)

In [10]:
weather['rain'] = ls_rain
weather['fog'] = ls_fog

In [11]:
status['date'] = pd.to_datetime(ls)

In [12]:
status['date'] = status['date'].astype(str)
weather['date'] = weather['date'].astype(str)

train = pd.merge(status, weather, on='date')

In [13]:
whole = train

In [15]:
whole['date'] = pd.to_datetime(whole['date'].tolist())

In [16]:
station = pd.read_csv('station.csv')

In [17]:
installation_ls_ = station['installation_date'].tolist()
installation_ls = []

for  s in installation_ls_:
    s_ = str(s)
    st = s_.split('/')
    installation_ls.append(str(st[2]) + '-' + str(st[0]) + '-' +str(st[1]))

installation_ls = pd.to_datetime(installation_ls)

In [18]:
delete_installation = []

for i in range(1226400):
    if whole.iloc[i,8] <= installation_ls[whole.iloc[i,5]]:
        delete_installation.append(1)
    else:   
        delete_installation.append(0)
            

In [19]:
whole['delete_installation'] = delete_installation

In [22]:
whole = whole[whole['delete_installation'] != 1]

In [24]:
whole = whole.drop(['delete_installation'], axis=1)

In [26]:
whole.columns

Index(['id', 'year', 'month', 'day', 'hour', 'station_id', 'bikes_available',
       'predict', 'date', 'holiday', 'weekday', 'max_temperature',
       'mean_temperature', 'min_temperature', 'max_dew_point',
       'mean_dew_point', 'min_dew_point', 'max_humidity', 'mean_humidity',
       'min_humidity', 'max_sea_level_pressure', 'mean_sea_level_pressure',
       'min_sea_level_pressure', 'max_visibility', 'mean_visibility',
       'min_visibility', 'max_wind_Speed', 'mean_wind_speed', 'precipitation',
       'cloud_cover', 'events', 'wind_dir_degrees', 'rain', 'fog'],
      dtype='object')

In [27]:
whole['weekday']

0          6
1          6
2          6
3          6
4          6
          ..
1226395    0
1226396    0
1226397    0
1226398    0
1226399    0
Name: weekday, Length: 1204704, dtype: int64

In [28]:
if whole.iloc[0,2] in [9,10]:
    print("a")

a


In [29]:
weekend_ls, spring_ls, summer_ls, fall_ls, winter_ls = [],[],[],[],[]

for i in range(1204704):
    if whole.iloc[i,10] == 0 or whole.iloc[i,10] == 6:
        weekend_ls.append(1)
    else:
        weekend_ls.append(0)

    if whole.iloc[i,2]  in [3,4,5]:
        spring_ls.append(1)
        summer_ls.append(0)
        fall_ls.append(0)
        winter_ls.append(0)
    elif whole.iloc[i,2]  in [6,7,8]:
        spring_ls.append(0)
        summer_ls.append(1)
        fall_ls.append(0)
        winter_ls.append(0)
    elif whole.iloc[i,2]  in [9,10,11]:
        spring_ls.append(0)
        summer_ls.append(0)
        fall_ls.append(1)
        winter_ls.append(0)
    else:
        spring_ls.append(0)
        summer_ls.append(0)
        fall_ls.append(0)
        winter_ls.append(1)

In [30]:
whole['spring'] = spring_ls
whole['summer'] = summer_ls
whole['fall'] = fall_ls
whole['winter'] = winter_ls

In [31]:
whole['weekend'] = weekend_ls

In [36]:
whole.columns

Index(['id', 'year', 'month', 'day', 'hour', 'station_id', 'bikes_available',
       'predict', 'date', 'holiday', 'weekday', 'max_temperature',
       'mean_temperature', 'min_temperature', 'max_dew_point',
       'mean_dew_point', 'min_dew_point', 'max_humidity', 'mean_humidity',
       'min_humidity', 'max_sea_level_pressure', 'mean_sea_level_pressure',
       'min_sea_level_pressure', 'max_visibility', 'mean_visibility',
       'min_visibility', 'max_wind_Speed', 'mean_wind_speed', 'precipitation',
       'cloud_cover', 'events', 'wind_dir_degrees', 'rain', 'fog', 'spring',
       'summer', 'fall', 'winter', 'weekend'],
      dtype='object')

In [37]:
station

,station_id,lat,long,dock_count,city,installation_date
0,0,37.32973,-121.90178,27,city1,8/6/2013
1,1,37.33070,-121.88898,15,city1,8/5/2013
2,2,37.33399,-121.89490,11,city1,8/6/2013
3,3,37.33141,-121.89320,19,city1,8/5/2013
4,4,37.33672,-121.89407,15,city1,8/7/2013
...,...,...,...,...,...,...
65,65,37.78963,-122.40081,27,city2,8/25/2013
66,66,37.35260,-121.90573,15,city1,12/31/2013
67,67,37.79854,-122.40086,15,city2,1/22/2014
68,68,37.49127,-122.23623,15,city3,2/20/2014


In [56]:
lat_ls, long_ls, city_ls, dock_ls = [],[],[],[]

for i in range(70):
    lat_ls.append(station.iloc[i,1])
    long_ls.append(station.iloc[i,2])
    dock_ls.append(station.iloc[i,3])
    if station.iloc[i,4] == 'city1':
        city_ls.append(1)
    elif station.iloc[i,4] == 'city2':
        city_ls.append(2)
    elif station.iloc[i,4] == 'city3':
        city_ls.append(3)
    elif station.iloc[i,4] == 'city4':
        city_ls.append(4)
    else:
        city_ls.append(5)

In [57]:
lat_ls_, long_ls_, city_ls_, dock_ls_ = [],[],[],[]

for i in range(1204704):
    j = whole.iloc[i,5]
    lat_ls_.append(lat_ls[j])
    long_ls_.append(long_ls[j])
    city_ls_.append(city_ls[j])
    dock_ls_.append(dock_ls[j])


In [58]:
whole['lat'] = lat_ls_
whole['long'] = long_ls_
whole['city'] = city_ls_
whole['dock'] = dock_ls_

In [60]:
whole

,id,year,month,day,hour,station_id,bikes_available,predict,date,holiday,...,fog,spring,summer,fall,winter,weekend,lat,long,city,dock
0,0,2013,9,1,0,0,11.0,0,2013-09-01,1,...,0,0,0,1,0,1,37.32973,-121.90178,1,27
1,1,2013,9,1,1,0,11.0,0,2013-09-01,1,...,0,0,0,1,0,1,37.32973,-121.90178,1,27
2,2,2013,9,1,2,0,11.0,0,2013-09-01,1,...,0,0,0,1,0,1,37.32973,-121.90178,1,27
3,3,2013,9,1,3,0,11.0,0,2013-09-01,1,...,0,0,0,1,0,1,37.32973,-121.90178,1,27
4,4,2013,9,1,4,0,11.0,0,2013-09-01,1,...,0,0,0,1,0,1,37.32973,-121.90178,1,27
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1226395,1226395,2015,8,31,19,69,8.0,0,2015-08-31,0,...,0,0,1,0,0,1,37.34273,-121.89562,1,15
1226396,1226396,2015,8,31,20,69,8.0,0,2015-08-31,0,...,0,0,1,0,0,1,37.34273,-121.89562,1,15
1226397,1226397,2015,8,31,21,69,8.0,0,2015-08-31,0,...,0,0,1,0,0,1,37.34273,-121.89562,1,15
1226398,1226398,2015,8,31,22,69,8.0,0,2015-08-31,0,...,0,0,1,0,0,1,37.34273,-121.89562,1,15


In [61]:
whole.to_csv('whole.csv')